In [ ]:
# Installation des dépendances
!pip install transformers torch accelerate bitsandbytes

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import warnings
warnings.filterwarnings("ignore")
# HF login to access Gemma 3 1B which is Open Weight but a gated model (need to ack the terms before accessing)
# Login your HuggingFace Account
from huggingface_hub import notebook_login
notebook_login()
# Execute and paste your token

In [ ]:
# Model Config
MODEL_NAME = "google/gemma-3-1b-it"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Used device: {device}")
print("Model Loading...")

# Model config
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    load_in_8bit=True if device == "cuda" else False,  # Quantification pour économiser la mémoire
)

# Pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Model loading succeed!")

def generate_response(prompt, max_length=128, temperature=0.7, top_p=0.9):
    """
    Generating answer
    """
    # Encodage du prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Génération de la réponse
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1
        )

    # decoding answer
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response[len(prompt):].strip()
    return response

def chat_loop():
    """
    interactive chat loop for testing purposes
    """
    print("\n👾 Gemma3-1b-it")
    print("=" * 25)
    print("type 'quit', 'exit' or 'bye' to leave")
    print("type 'clear' to wipe history")
    print("=" * 25)

    conversation_history = []

    while True:
        try:
            # User input
            user_input = input("\n👽 You: ").strip()

            # Commandes spéciales
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("\n👋 Bye!")
                break

            if user_input.lower() == 'clear':
                conversation_history = []
                print("🧹 history wiped!")
                continue

            if not user_input:
                continue

            # Construction du prompt avec l'historique
            if conversation_history:
                context = "\n".join(conversation_history[-6:])  # Garder les 6 derniers échanges
                prompt = f"{context}\nUser: {user_input}\nAssistant:"
            else:
                prompt = f"User: {user_input}\nAssistant:"

            print("\n👾 Gemma3-1b-it:", end=" ", flush=True)

            # Génération de la réponse
            response = generate_response(
                prompt,
                max_length=1024,
                temperature=0.7,
                top_p=0.9
            )

            print(response)

            # Mise à jour de l'historique
            conversation_history.append(f"Utilisateur: {user_input}")
            conversation_history.append(f"Assistant: {response}")

        except KeyboardInterrupt:
            print("\n\n⚠️ interruption detected. type 'quit' to leave properly.")
            continue
        except Exception as e:
            print(f"\n❌ Error: {str(e)}")
            continue

# Fonction pour tester le modèle avec un prompt simple
def test_model():
    """
    Model Testing
    """
    test_prompt = "Hi, How can I help you? Feel free to ask me about what is Project Zero and how does it contribute to security? or How does Google ensure the security of its cloud services?"
    print(f"\n👾 Gemma3-1b-it: '{test_prompt}'")
    response = generate_response(test_prompt, max_length=100)
    print(f"👾 Answer: {response}")

# Lancement du test puis du chat
if __name__ == "__main__":
    test_model()
    chat_loop()